In [ ]:
%%capture
import pandas as pd
# from pandas_profiling import ProfileReport
import numpy as np
import re

In [ ]:
# Đọc file Json, chuyển từ CSDL SQL trong MySQL sang File Json ổn định hơn và ít bị lỗi như chuyển sang File csv
df_sv = pd.read_json('raw_data/uit_sinhvien.json')
df_diem = pd.read_json('raw_data/uit_diem.json')
df_monhoc = pd.read_json('raw_data/uit_monhoc.json')

In [ ]:
df_sv

# I. Tạo ra cột mới chứa giá trị là Monhocdaidien  dùng để thay thế tên mamh đang hiện hành phía trước 

In [ ]:
#Những ô ko có gtri trong csdl này mà lại ko mang GT NULL=> mang giá trị ''
# Xử lý các môn học tương đương trước vì các môn học cũ thì chắc chắn đã đóng.
# List1 các môn học Không bị đóng trong toàn df_monhoc
df_list1=df_monhoc[df_monhoc['trangthai'] ==1]
List1=df_list1['mamh'].to_list()
List1

In [ ]:
def Tao_listmontd(mondangxet,montuongduong):
    List2=montuongduong.split(',')
    if List2==['']:
        List2=[mondangxet]
        return List2
    List2.append(mondangxet)
    return List2

In [ ]:
# Gom nhóm môn học đang xét và các môn học tương đương vào 1 list2
# Phương thức agg ở đây mang ý nghĩa tổng hợp,axis=1 tổng hợp theo cột đối với 1 phần tử là hàng. 
# Từ 2 cột tổng hợp đưa ra được 1 cột mới
df_monhoc['List_montd']=df_monhoc.agg(lambda x: Tao_listmontd(x['mamh'],x['mh_td']),axis=1)
df_monhoc['List_montd']


In [ ]:
def Chon_monconmo(List2):
    #Cần lưu ý các môn như IS214,IS101
    #len(List2)==1 Trường hợp môn này ko có môn tương đương. Vì nếu ko có môn tương đương thì cũng có tên của chính nó trong List2.
    if len(List2)!=1:
        def fill_monconhoc(MoncanKT):
            return (MoncanKT in List1 )

        List3 = list(filter(lambda x: fill_monconhoc(x),List2))
        # Chọn thằng đứng đầu
        List3.sort()
        # Nếu List3 == [] chứng tỏ môn này có môn tương đương. Nhưng tất cả các môn tương đương đều bị đóng
        if List3 != []:
            return List3[0]
        # Chọn 1 thằng đại diện, ta đã biết được là các môn trong List2 này ko môn nào còn mở.
        List2.sort()
        return List2[0]
        
    return List2[0]


In [ ]:
# So sánh list1 vs list2 => list3 các môn chưa bị đóng trong nhóm list2
# Sort list3 và chọn thằng đứng đầu (do thứ tự nó là chữ cái đứng đầu). Môn học đại diện S
# Lưu ý xử lý trường hợp cả mã môn mới và mã môn cũ đề là những môn đã đóng lớp.
df_monhoc['Mondaidien']=df_monhoc.agg(lambda x: Chon_monconmo(x['List_montd']),axis=1)
df_monhoc['Mondaidien']


In [ ]:
Dict1={}
def Tao_Dict1(mamhcu,Mondaidien,Dict1):
    # mamhcu là khóa vì dùng nó để giải quyết thay thế cho các môn học đã cũ và khóa là duy nhất
    List_mamhcu=mamhcu.split(',')
    for i in List_mamhcu:
        Dict1[i]=Mondaidien
    return

In [ ]:
# Giờ mới xử các môn học đã cũ. Vì có TH các mã môn học đã cũ, ko có môn học tương đương tuy nhiên giờ đã có mã mới.
# VD: trường hợp của môn PHY12
# Lọc Dict1 các môn nằm trong cột mamhcu ánh xạ với mã của môn đại diện.
df_Dict1=df_monhoc[df_monhoc['mamhcu'] !='']
df_Dict1.agg(lambda x: Tao_Dict1(x['mamhcu'],x['Mondaidien'],Dict1),axis=1)
Dict1



In [ ]:
# print(len(Dict1.keys()))
def Doi_mamhcu(x):
    if x in Dict1.keys():
        x=Dict1[x]
    return x

In [ ]:
# Áp dúng từ điển Dict1 để thay thế các môn cũ đang nằm trong cột Mondaidien của df_monhoc.
df_monhoc['Mondaidien']=df_monhoc['Mondaidien'].apply(Doi_mamhcu)
df_monhoc


In [ ]:
df_monhoc.columns

In [ ]:
# Đổi lại thuộc tính hệ đào tạo cho ko bị nhầm.
df_monhoc.columns=['id', 'mamh', 'tenmh', 'tenta', 'loaimh', 'sotinchi', 'sotc_lt',
       'sotc_th', 'hedt_monhoc', 'khoaql', 'hinhthucth', 'mamhcu', 'mh_td',
       'mh_truoc', 'mh_tq', 'tldanhgia_qt', 'tldanhgia_th', 'tldanhgia_gk',
       'tldanhgia_ck', 'htthi_lt_gk', 'htthi_lt_gk_tg', 'htthi_lt_ck',
       'htthi_lt_ck_tg', 'htthi_th_ck', 'trangthai', 'ngonngu', 'ghichu',
       'dcid', 'List_montd', 'Mondaidien']
df_monhoc.columns

In [ ]:
df_sv.columns

In [ ]:
df_sv.columns=['id', 'masv', 'uid', 'hoten', 'ngaysinh', 'gioitinh', 'noisinh', 'cmnd',
       'cmnd_ngaycap', 'cmnd_noicap', 'lopsh', 'khoa', 'hedt_sv', 'khoahoc',
       'chuyennganh', 'chuyennganh2', 'nganh_tt', 'tinhtrang', 'email',
       'email_canhan', 'email_alias', 'dienthoai', 'dantoc', 'tongiao',
       'xuatthan', 'doituong', 'doituong_chitiet', 'khuvuc', 'vungmien',
       'chinhsach']
df_sv.columns

# Chọn những thuộc tính cần thiết để bổ trợ thông tin cho một môn học.
Bước này và các bước sau nên sắp lại thứ tự của các thuộc tính.

In [ ]:
#  sotc_lt  //monhoc
#  sotc_th  //monhoc
#  hinhthucth:  // monhoc
#  khoaql: // monhoc
#  hedt  // monhoc
#  ngonngu: // monhoc
#  mamh: dùng để merge với bảng điểm
#  Mondaidien: dùng để thay thế các mamh sau khi merge với bảng điểm

df_monhoc_selected=df_monhoc[['mamh','Mondaidien','sotc_lt','sotc_th','hinhthucth','khoaql','hedt_monhoc','ngonngu']]
df_monhoc_selected['hedt_monhoc']=df_monhoc_selected['hedt_monhoc'].str.upper()
df_monhoc_selected['hinhthucth']=df_monhoc_selected['hinhthucth'].map('hinhthucth{}'.format, na_action='ignore')
df_monhoc_selected

In [ ]:
df_monhoc_selected = df_monhoc_selected.astype({'hinhthucth': str})
df_monhoc_selected.dtypes  

# Chọn những thuộc tính cần thiết của bảng điểm

In [ ]:
# + diem mh
# + sotc 
# + hocky của mamh theo thực tế năm nhất: 1,2,3
# + trangthai:
# mamh: merge với bảng điểm và còn sd sau
# masv merge với bảng sinh viên
df_diem_selected=df_diem[['masv','mamh','diem','sotc','hocky','namhoc','trangthai']]
df_diem_selected['hocky']=df_diem_selected['hocky'].map('hk{}'.format, na_action='ignore')
df_diem_selected['trangthai']=df_diem_selected['trangthai'].map('trangthai{}'.format, na_action='ignore')
df_diem_selected

In [ ]:
df_diem_selected = df_diem_selected.astype({"hocky": str, "trangthai": str})
df_diem_selected.dtypes  

# Merge bảng điểm và bảng môn học

In [ ]:
df_diem_monhoc=pd.merge(df_diem_selected,df_monhoc_selected,on='mamh')
# Đã bỏ mamh
df_diem_monhoc=df_diem_monhoc[['masv','Mondaidien','diem','sotc','trangthai'
                               ,'sotc_lt','sotc_th','hinhthucth','ngonngu'
                               ,'khoaql','hedt_monhoc','hocky','namhoc']]


In [ ]:
# df_diem_monhoc.to_excel("df_diem_monhoc.xlsx")  

# Thay đổi mã môn học bằng tên môn học đại diện

In [ ]:
# Có thể sd cách này nhưng tiện thể bước này đã xử lý ở đoạn trên rồi.
# df_diem_monhoc=df_diem_monhoc.drop(columns='mamh')
# df_diem_monhoc

# Chọn thuộc tính bảng SV và Merge bảng sv và bảng diem_monhoc

In [ ]:
# 1.masv
# 2.ngaysinh  
# 3.gioitinh 
# 4.noisinh 
# 5.cmnd_noicap
# 6.cmnd_ngaycap
# 7.lopsh
# 8.khoa
# 9.hedt
# 10.khoahoc
# 11.chuyennganh
# 12.chuyennganh2
# 13.nganh_tt
# 14.dantoc
# 15.tongiao
# 16.xuatthan
# 17.doituong
# 18.doituong_chitiet
# 19.khuvuc
df_sv_selected=df_sv[['masv','gioitinh','noisinh','ngaysinh','cmnd_noicap','cmnd_ngaycap'
                      ,'lopsh','khoa','hedt_sv','khoahoc','chuyennganh','chuyennganh2','nganh_tt','dantoc','tongiao',
                     'xuatthan','doituong','doituong_chitiet','khuvuc']]

df_sv_selected['gioitinh']=df_sv_selected['gioitinh'].replace({1:'nam',0:'nu'})
df_sv_selected['dantoc']=df_sv_selected['dantoc'].map('dantoc{}'.format, na_action='ignore')
df_sv_selected['tongiao']=df_sv_selected['tongiao'].map('tongiao{}'.format, na_action='ignore')
df_sv_selected['xuatthan']=df_sv_selected['xuatthan'].map('xuatthan{}'.format, na_action='ignore')

df_sv_selected

In [ ]:
# df_sv_selected.to_excel('df_sv_selected.xlsx')
df_sv_selected = df_sv_selected.astype({"gioitinh": str, "dantoc": str, "tongiao": str,"xuatthan": str})
df_sv_selected.dtypes  

In [ ]:
df_tonghop3bang=pd.merge(df_sv_selected,df_diem_monhoc,on='masv')
df_tonghop3bang

In [ ]:
# df_tonghop3bang.to_excel('df_tonghop3bang.xlsx')

# Lọc ra những dòng thuộc năm nhất và năm 2 của các sinh viên Khóa dùng để huấn luyện.

In [ ]:
# Cho phép chọn khóa dùng để huần luyện.
list_cackhoa=df_tonghop3bang['khoahoc'].unique()
soluongcackhoa=df_tonghop3bang['khoahoc'].nunique()
print(list_cackhoa)
print(soluongcackhoa)

In [ ]:
# Nhập các khóa cần dùng để huấn luyện
cackhoa_canchon=[]
print('Stop:y/n')
for i in range(soluongcackhoa):
    khoa=input()
    if khoa=='y':
        break
    cackhoa_canchon.append(int(khoa))


In [ ]:
 cackhoa_canchon

In [ ]:
Nam1={}
#Tạo quy đổi giữa khóa và năm học thứ nhất, năm học thứ 2.
def Quydoi_khoa_nam1(khoa):
    khoa=int(khoa)
    return 2006+(khoa-1)

for khoa in cackhoa_canchon:
    Nam1[khoa]=Quydoi_khoa_nam1(khoa)
Nam1

In [ ]:
#Chỉ lấy thông tin sau năm nhất và năm hai của sinh viên các khóa được chọn

df_nam12_all=pd.DataFrame()


for khoa in cackhoa_canchon:
    # Lấy thông tin 2 năm học đầu tiên ứng với sinh viên từng Khóa.
    df_nam12=df_tonghop3bang[(df_tonghop3bang.khoahoc==khoa)&
                             (df_tonghop3bang.namhoc < Nam1[khoa]+2)
                           &(df_tonghop3bang.namhoc > Nam1[khoa]-1)]# so sánh như vầy để ngăn cản TH 2 SV lớp MTLK2018 học môn của năm học 2017 => mâu thuẫn    
    # và TH sinh viên ko có dữ liệu năm nhất nhưng có học năm 2.=> bỏ ra ngay.

    # Gộp vào dl nam1,2 của các khóa được chọn , ignore_index có thể gây ra tình trạng chỉ số bị lặp lại từ 0
    df_nam12_all=pd.concat([df_nam12_all,df_nam12],ignore_index=True)



In [ ]:
df_nam12_all

In [ ]:
# df_nam12_all.to_excel('df_nam12_all.xlsx')

# Tạo 2 bảng: (Tính Tổng số TC nam 1, DTB nam 1, Loại học lực nam 1) và (nhãn của bài toán)

In [ ]:
df_mienthi=pd.DataFrame()
# Tách th miễn thi và hoãn thi => Để tính số tc cộng thêm thôi.
df_mienthi=df_nam12_all[(df_nam12_all['trangthai']=='trangthai4')|(df_nam12_all['trangthai']=='trangthai5')]
df_mienthi=df_mienthi.groupby(['masv','khoahoc','namhoc'], as_index=False).aggregate(sotc_thdb=pd.NamedAgg(column="sotc", aggfunc="sum"))
df_mienthi


In [ ]:
# Tách th không miễn thi và hoãn thi
df_muctieu1=pd.DataFrame()
df_muctieu1=df_nam12_all[(df_nam12_all.trangthai!='trangthai0') & (df_nam12_all.trangthai!='trangthai4')
                         &(df_nam12_all.trangthai!='trangthai5')]


df_muctieu1['Diem*Tc']= df_muctieu1.agg(lambda x: x['diem']*x['sotc'],axis=1)

df_muctieu1=df_muctieu1.groupby(['masv','khoahoc','namhoc'], as_index=False).aggregate({'Diem*Tc':'sum', 'sotc': 'sum'})

In [ ]:
# Chỉ lấy những sinh viên có cả thông tin học của năm nhất và năm 2.
Stats_masv=df_muctieu1['masv'].value_counts()
Stats_masv=Stats_masv[Stats_masv==2]
Stats_masv=Stats_masv.index
# type(Stats_masv)
len(Stats_masv)

In [ ]:
df_muctieu1=df_muctieu1[df_muctieu1['masv'].isin(Stats_masv)]
df_muctieu1

In [ ]:
def Tinh_DTB(Diem_Tc,Tongsotc):
    if Tongsotc==0:
        return 0
    return (Diem_Tc/Tongsotc)
# Để dưới dạng số để có thể xác định là có rớt hạng hay không.
def Tinh_Loaihocluc(DTBnam):
    if DTBnam>=7.5:
        return 2
    elif DTBnam>=6.0:
        return 1
    else:
        return 0
    

In [ ]:
df_muctieu1['DTBnam']=df_muctieu1.agg(lambda x: Tinh_DTB(x['Diem*Tc'],x['sotc']),axis=1)
df_muctieu1['Loaihoclucnam']=df_muctieu1.agg(lambda x: Tinh_Loaihocluc(x['DTBnam']),axis=1)
df_muctieu1

In [ ]:
# Lọc ra Tổng số TC nam 1, DTB nam 1, Loại học lực nam 1
df_tc_dtb_hocluc_1=pd.DataFrame()
# Chọn điểm năm nhất ứng với từng khóa
for khoa in cackhoa_canchon:
    subdf_tc_dtb_hocluc_1=df_muctieu1[(df_muctieu1.namhoc== Nam1[khoa])
                               & (df_muctieu1.khoahoc== khoa)]
    
    df_tc_dtb_hocluc_1=pd.concat([df_tc_dtb_hocluc_1,subdf_tc_dtb_hocluc_1],ignore_index=True)
df_tc_dtb_hocluc_1


In [ ]:
# Tạo bảng lưu tổng Diem*sotc mục đích để tính lại điểm điền vào những môn thiếu sau này.
df_dienthieu_diem=df_tc_dtb_hocluc_1[['masv','Diem*Tc']]
df_dienthieu_diem

In [ ]:
def Daodau_DTBnam(x):
    if (x['namhoc']== Nam1[x['khoahoc']]) and x['DTBnam']>=0 :
        x['DTBnam']= -x['DTBnam']
    return x['DTBnam']

def Daodau_Loaihoclucnam(x):
    if (x['namhoc']== Nam1[x['khoahoc']]) and x['Loaihoclucnam']>=0 :
        x['Loaihoclucnam']= 0
    return x['Loaihoclucnam']

df_muctieu1['DTBnam']=df_muctieu1.agg(lambda x: Daodau_DTBnam(x),axis=1)

df_muctieu1['Loaihoclucnam']=df_muctieu1.agg(lambda x: Daodau_Loaihoclucnam(x),axis=1)

df_muctieu1

In [ ]:
df_muctieu1=df_muctieu1.groupby(['masv','khoahoc'], as_index=False).aggregate({'DTBnam':'sum', 'Loaihoclucnam': 'sum'})
df_muctieu1

In [ ]:
def Tao_nhan(x):
    return x['Loaihoclucnam']

df_muctieu1['Nhan_xuhuong']=df_muctieu1.agg(Tao_nhan,axis=1)
df_muctieu1

In [ ]:
# Kết hợp df_tc_dtb_hocluc_1 và df_muctieu1 để tạo ra (Tổng tc,dtb,loại học lực) năm 1 và nhãn dữ liệu 
df_muctieu1=df_muctieu1.drop(columns=['DTBnam','Loaihoclucnam'])
df_tc_dtb_hocluc_1=df_tc_dtb_hocluc_1.drop(columns=['namhoc','Diem*Tc'])

In [ ]:
df_muctieu1=pd.merge(df_tc_dtb_hocluc_1,df_muctieu1,on=['masv','khoahoc'])
# Đổi tên cột
df_muctieu1.columns=['masv','khoahoc','Tong_sotc','DTBnam'
                               ,'Loaihoclucnam','Nhan_xuhuong']
df_muctieu1

In [ ]:
# Chọn điểm năm nhất ứng với từng khóa đối với các TH đặc biệt: Miễn thi, hoãn thi, hủy thi.
df_mienthi_nam1=pd.DataFrame()
for khoa in cackhoa_canchon:
    subdf_mienthi=df_mienthi[(df_mienthi.namhoc== Nam1[khoa])
                               & (df_mienthi.khoahoc== khoa)]
    
    df_mienthi_nam1=pd.concat([df_mienthi_nam1,subdf_mienthi],ignore_index=True)
df_mienthi_nam1

In [ ]:
# Merge lại để cộng số tín chỉ hoãn thi, miễn thi vào tổng tín chỉ
df_muctieu1=pd.merge(df_muctieu1,df_mienthi_nam1,on=['masv','khoahoc'],how='left')
df_muctieu1=df_muctieu1.drop(columns='namhoc')

df_muctieu1['Tong_sotc'] = np.where(df_muctieu1.sotc_thdb.isnull(),
                        df_muctieu1.Tong_sotc, 
                        df_muctieu1.Tong_sotc+df_muctieu1.sotc_thdb)

# df_muctieu1 = df_muctieu1.astype({"Tong_sotc": int})

df_muctieu1=df_muctieu1.drop(columns='sotc_thdb')
df_muctieu1

In [ ]:
# df_muctieu1.to_excel('df_muctieu1.xlsx')

# Tạo 2 bảng: Mục đích đầu tiên là tính được điểm HK1 năm 1 và HK2 năm 1 sau đó sẽ => Tạo nhãn Tổng số TC HK1, DTB HK1, Loại học lực HK1, Tổng số TC HK2, DTB HK2, Loại học lực HK2, Điểm HK2- điểm HK1,Hạng HK2- hạng Hk1.

In [ ]:
df_nam1=pd.DataFrame()

# Chọn điểm năm nhất ứng với từng khóa và chọn những SV có dữ liệu cả năm 1 và năm 2
for khoa in cackhoa_canchon:
    subdf_nam1=df_nam12_all[(df_nam12_all.namhoc== Nam1[khoa])
                               & (df_nam12_all.khoahoc== khoa)
                           & (df_nam12_all['masv'].isin(Stats_masv)) ]
    df_nam1=pd.concat([df_nam1,subdf_nam1],ignore_index=True)
df_nam1

In [ ]:
df_nam1.columns

In [ ]:
# Học kỳ 3 gộp với HK2
df_muctieu2=df_nam1.copy()
df_muctieu2['hocky']=df_muctieu2['hocky'].replace({'hk3':'hk2'})
# Lọc những môn hoãn thi, miễn thi ra bảng riêng. Theo từng học kỳ
df_mienthi_hk=pd.DataFrame()

df_mienthi_hk=df_muctieu2[(df_muctieu2['trangthai']=='trangthai4')|(df_muctieu2['trangthai']=='trangthai5')]

df_mienthi_hk=df_mienthi_hk.groupby(['masv','hocky','khoahoc'], as_index=False).aggregate(
    sotc_thdb=pd.NamedAgg(column="sotc", aggfunc="sum"))
df_mienthi_hk

In [ ]:
df_muctieu2=df_muctieu2[(df_muctieu2.trangthai!='trangthai0')&(df_muctieu2.trangthai!='trangthai4')&
                        (df_muctieu2.trangthai!='trangthai5')]

In [ ]:
df_muctieu2['Diem*Tc']= df_muctieu2.agg(lambda x: x['diem']*x['sotc'],axis=1)
df_muctieu2=df_muctieu2.groupby(['masv','hocky','khoahoc'], as_index=False).aggregate({'Diem*Tc':'sum', 'sotc': 'sum'})
df_muctieu2

In [ ]:
df_muctieu2['DTB_HK']=df_muctieu2.agg(lambda x: Tinh_DTB(x['Diem*Tc'],x['sotc']),axis=1)
df_muctieu2['Loaihocluc_HK']=df_muctieu2.agg(lambda x: Tinh_Loaihocluc(x['DTB_HK']),axis=1)
df_muctieu2

In [ ]:
# Merge lại để cộng số tín chỉ hoãn thi, miễn thi vào tổng tín chỉ
df_muctieu2=pd.merge(df_muctieu2,df_mienthi_hk,on=['masv','hocky','khoahoc'],how='left')




df_muctieu2['sotc'] = np.where(df_muctieu2.sotc_thdb.isnull(),
                        df_muctieu2.sotc, 
                        df_muctieu2.sotc+df_muctieu2.sotc_thdb)


df_muctieu2=df_muctieu2.drop(columns='sotc_thdb')
df_muctieu2

In [ ]:
type(df_muctieu2['hocky'][0])

In [ ]:
df_muctieu2['hocky'].value_counts()

In [ ]:
# Đảo dấu để tính chênh lệch 2 HK.
# Tạo 1 bảng mới Mục tiêu 3. Lưu ý ko được gán mà phải copy
df_muctieu3=df_muctieu2.copy()

def Daodau_DTB_hk(x):
    if x['hocky']== 'hk1' and x['DTB_HK']>=0 :
        x['DTB_HK']= -x['DTB_HK']
    return x['DTB_HK']

def Daodau_Loaihocluc_hk(x):
    if x['hocky']== 'hk1' and x['Loaihocluc_HK']>=0 :
        x['Loaihocluc_HK']= -x['Loaihocluc_HK']
    return x['Loaihocluc_HK']

df_muctieu3['DTB_HK']=df_muctieu3.agg(lambda x: Daodau_DTB_hk(x),axis=1)

df_muctieu3['Loaihocluc_HK']=df_muctieu3.agg(lambda x: Daodau_Loaihocluc_hk(x),axis=1)

df_muctieu3

In [ ]:
type(df_muctieu3['Loaihocluc_HK'][0])

In [ ]:
# df_muctieu2
df_muctieu3=df_muctieu3.groupby(['masv','khoahoc'], as_index=False).aggregate(    
    Diff_DTB_HK=pd.NamedAgg(column="DTB_HK", aggfunc="sum"),
    Diff_Loaihocluc_HK=pd.NamedAgg(column="Loaihocluc_HK", aggfunc="sum"))

df_muctieu3

In [ ]:
# Tổng số TC HK1, DTB HK1, Loại học lực HK1, Tổng số TC HK2, DTB HK2, Loại học lực HK2, Điểm HK2- điểm HK1,Hạng HK2- hạng Hk1.
df_muctieu2=df_muctieu2.groupby(['masv','khoahoc'], as_index=False).aggregate({'hocky':list,'sotc':list,'DTB_HK': list,
                                                                              'Loaihocluc_HK':list})

In [ ]:
df_muctieu2

In [ ]:
df_muctieu2['hocky'].value_counts()

In [ ]:
type(df_muctieu2['DTB_HK'][0][1])

In [ ]:
# Tạo tên các cột
column_names_df_muctieu2 = ['masv', 'khoahoc','DTB_HK1','Sotc_HK1','Loaihocluc_HK1','DTB_HK2','Sotc_HK2','Loaihocluc_HK2']

def Tao_df_muctieu2Final(x):
    # Gán dữ liệu cho List
    A_row=[x['masv'],x['khoahoc']]
    if len(x['hocky'])==2:
        for i in range(len(x['hocky'])):
            A_row.append(float(x['DTB_HK'][i]))
            A_row.append(int(x['sotc'][i]))
            A_row.append(int(x['Loaihocluc_HK'][i]))
    elif len(x['hocky'])==1:
        #Nếu có sửa đổi giá trị thể hiện của Học kỳ thì sửa tại đây thôi.
        if x['hocky'][0] =='hk1':
            A_row.append(float(x['DTB_HK'][0]))
            A_row.append(int(x['sotc'][0]))
            A_row.append(int(x['Loaihocluc_HK'][0]))
            A_row.extend([None,None,None])
        else:
            A_row.extend([None,None,None])
            A_row.append(float(x['DTB_HK'][0]))
            A_row.append(int(x['sotc'][0]))
            A_row.append(int(x['Loaihocluc_HK'][0]))
    else:
        A_row.extend([None,None,None,None,None,None])
    
    # Tạo ra Series
    output_series = pd.Series(A_row, index=column_names_df_muctieu2)
    return output_series

In [ ]:
df_muctieu2Final=df_muctieu2.apply(Tao_df_muctieu2Final,axis=1)
df_muctieu2Final

In [ ]:
# Lưu ý nếu bước này ta ép kiểu int cho những cột có chứa giá trị thiếu thì có thể gây ra lỗi. Nên để Pandas xử lý tự động.
# Trong hàm Tao_df_muctieu2Final ta đã ép kiểu cho từng phần tử trước khi tạo Series, 
# nên từ đó Pandas sẽ tự xử lý để đưa ra kiểu dữ liệu phù hợp với từng cột
# df_muctieu2Final = df_muctieu2Final.astype({"Sotc_HK2": int, "Loaihocluc_HK2": int})
df_muctieu2Final.dtypes  
# df_muctieu2Final.to_excel('df_muctieu2Final.xlsx')

In [ ]:
df_muctieu2Final=pd.merge(df_muctieu2Final,df_muctieu3,on=['masv','khoahoc'])
df_muctieu2Final

# Tạo bảng riêng=> mục đích là tạo thành các thuộc tính môn dựa vào thống kê SV học nhiều môn nhất.

In [ ]:
# Dựa trên bảng điểm các môn năm nhất thôi. Đếm giá trị MSSV nào xuất hiện nhiều nhất trong bảng.
Stats_masv=df_nam1['masv'].value_counts()
Stats_masv

In [ ]:
Somon_canlay=Stats_masv[0]

In [ ]:
Stats_masv.value_counts()

In [ ]:
df_nam1.columns

In [ ]:
# Bỏ bớt các thuộc tính ko cần thiết và Groupby bảng df_nam1 để nhóm các mã môn,... vào từng list
df_nam1_sub1=df_nam1.groupby(['masv','khoahoc'], as_index=False).aggregate({'Mondaidien':list, 
                                                                            'diem':list,
                                                                            'sotc': list,
                                                                            'trangthai': list,
                                                                            'sotc_lt': list,
                                                                            'sotc_th': list,
                                                                            'hinhthucth': list,
                                                                            'ngonngu': list,
                                                                            'khoaql': list,
                                                                            'hedt_monhoc': list,
                                                                            'hocky': list,
                                                                           })

df_nam1_sub1



In [ ]:
# Tạo tên các cột
column_names = ["masv", "khoahoc"]
for i in range(Somon_canlay):
    mamh_='mamh_'+str(i+1)
    diemmon_='diemmon_'+str(i+1)
    sotcmon_='sotcmon_'+str(i+1)
    trangthaimon_='trangthaimon_'+str(i+1)
    sotc_ltmon_='sotc_ltmon_'+str(i+1)
    sotc_thmon_='sotc_thmon_'+str(i+1)
    hinhthucthmon_='hinhthucthmon_'+str(i+1)
    ngonngumon_='ngonngumon_'+str(i+1)
    khoaqlmon_='khoaqlmon_'+str(i+1)
    hedt_monhoc_='hedt_monhoc_'+str(i+1)
    hocky_mon_='hocky_mon_'+str(i+1)
    column_names.extend([mamh_,diemmon_,sotcmon_,trangthaimon_,
                        sotc_ltmon_,sotc_thmon_,hinhthucthmon_,ngonngumon_,
                        khoaqlmon_,hedt_monhoc_,hocky_mon_])
def Tao_df_muctieu4(x):
    # Gán dữ liệu cho List
    A_row=[x['masv'],x['khoahoc']]
    
    for i in range(len(x['Mondaidien'])):
        A_row.append(x['Mondaidien'][i])
        A_row.append(x['diem'][i])
        A_row.append(x['sotc'][i])
        A_row.append(x['trangthai'][i])
        A_row.append(x['sotc_lt'][i])
        A_row.append(x['sotc_th'][i])
        A_row.append(x['hinhthucth'][i])
        A_row.append(x['ngonngu'][i])
        A_row.append(x['khoaql'][i])
        A_row.append(x['hedt_monhoc'][i])
        A_row.append(x['hocky'][i])
    
    #Những cột còn thiếu so với khuôn dữ liệu chuẩn đã tạo ra     
    So_conthieu=len(column_names)-len(A_row)
    for i in range(So_conthieu):
        A_row.append('')
    # Tạo ra Series
    output_series = pd.Series(A_row, index=column_names)
    return output_series

In [ ]:
# df_nam1_sub1.to_excel('df_nam1_sub1.xlsx')
# Từ từng list ta sao chép các phần tử qua bảng mới.
df_muctieu4=df_nam1_sub1.apply(Tao_df_muctieu4,axis=1)
df_muctieu4

In [ ]:
# df_muctieu4.to_excel('df_muctieu4.xlsx')

# Tổng hợp lại để đủ 239 thuộc tính+ 1 nhãn

In [ ]:
# df_muctieu4 ,df_muctieu2Final ,df_muctieu1
df_Final=pd.merge(df_muctieu4,df_muctieu2Final,on=['masv','khoahoc'])
df_Final=pd.merge(df_Final,df_muctieu1,on=['masv','khoahoc'])
df_Final=pd.merge(df_sv_selected,df_Final,on=['masv','khoahoc'])
df_Final

In [ ]:
# # Xóa đối tượng chi tiết
df_Final=df_Final.drop(columns='doituong_chitiet')

def Laynam(x):
    if x=='':
        return 0
    else:
        return int(x[0:4])
df_Final['ngaysinh']=df_Final['ngaysinh'].map(lambda x: Laynam(x))
df_Final['cmnd_ngaycap']=df_Final['cmnd_ngaycap'].map(lambda x: Laynam(x))

def Tinhtuoi_cmnd(x):
    output=x['cmnd_ngaycap']-x['ngaysinh']
    if output<=0:
        return 0
    else:
        return output
df_Final['cmnd_ngaycap']=df_Final.agg(lambda x: Tinhtuoi_cmnd(x),axis=1)

def Tinhtuoi_vaoDH(x):
    output=Nam1[int(x['khoahoc'])]-x['ngaysinh'] 
    if output<=0:
        return 0
    else:
        return output
df_Final['ngaysinh']=df_Final.agg(lambda x: Tinhtuoi_vaoDH(x),axis=1)

df_Final=df_Final.rename(columns={"ngaysinh": "TuoivaoDH", "cmnd_ngaycap": "TuoicapCMND"})

df_Final

In [ ]:
# Chuẩn hóa lớp sh
def Chuanhoa_lopsh(x):
    output = ''.join([i for i in x if not i.isdigit()])
    return output
df_Final['lopsh']=df_Final['lopsh'].map(lambda x: Chuanhoa_lopsh(x))


# df_Final.to_excel('df_Final.xlsx')

In [ ]:
df_Final_dtypes=df_Final.dtypes.to_dict()

In [ ]:
# Loại học lực thì int cũng được
df_Final_dtypes

# 1. Làm sạch

In [ ]:
# Làm sạch thông tin chung
def no_accent_vietnamese(s):
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'I', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    return s


In [ ]:
df_Final['noisinh']=df_Final['noisinh'].str.lower().str.normalize('NFKC')
df_Final['cmnd_noicap']=df_Final['cmnd_noicap'].str.lower().str.normalize('NFKC')

def Xuly_kytu(x):
    if x.startswith('tỉnh '):
        x = x[5:]
    if x.startswith('thành phố '):
        x = x[10:]
    if x.startswith('tp.'):
        x = x[3:]
    if x.endswith('city'):
        x = x[:-4]
    x=no_accent_vietnamese(x)
    
    if x.find('ho') != -1 and x.find('chi') != -1 and x.find('minh') != -1:
        x='hochiminh'
    if x.find('dac lac') != -1 or x.find('daklak')!= -1 :
        x='dak lak'
    if x.find('dac nong') != -1 or x.find('daknong')!= -1:
        x='dak nong'
    
    return x
df_Final['noisinh'] = df_Final['noisinh'].map(Xuly_kytu)
df_Final['cmnd_noicap'] = df_Final['cmnd_noicap'].map(Xuly_kytu)


mode_noisinh=df_Final.noisinh.mode()[0]


In [ ]:
def Impute_noisinh(x):
    global mode_noisinh
    if x['noisinh']==''and x['cmnd_noicap']!='':
        x['noisinh']=x['cmnd_noicap']
        return x['noisinh']
    if x['noisinh']=='' and x['cmnd_noicap']=='':
        return mode_noisinh

    return x['noisinh']
def Impute_cmnd_noicap(x):
    if x['noisinh']!=''and x['cmnd_noicap']=='':
        x['cmnd_noicap']=x['noisinh']
        return x['cmnd_noicap']
    
    return x['cmnd_noicap']


df_Final['noisinh']=df_Final.apply(Impute_noisinh,axis=1)
df_Final['cmnd_noicap']=df_Final.apply(Impute_cmnd_noicap,axis=1)


In [ ]:
def Impute_thongtinchung_conlai(x):
    if x['gioitinh']=="":
        x['gioitinh']=df_Final.gioitinh.mode()[0]
    if x['TuoivaoDH']=="":
        x['TuoivaoDH']=df_Final.TuoivaoDH.mode()[0]
    if x['TuoicapCMND']=="":
        x['TuoicapCMND']=df_Final.TuoicapCMND.mode()[0]
    if x['TuoicapCMND']<14:
        x['TuoicapCMND']=14
    if x['nganh_tt']=="":
        x['nganh_tt']='khongco'
    if x['doituong']=="":
        x['doituong']='khongco'
    return x
    
df_Final=df_Final.apply(Impute_thongtinchung_conlai,axis=1)


In [ ]:
# df_Final.to_excel('df_Final1.xlsx')

# Tính số tín chỉ quy định tại mỗi học 

In [ ]:
df_ctdaotao = pd.read_json('raw_data/uit_ctdaotao.json')
df_ctdaotao

In [ ]:
df_ctdaotao_selected=df_ctdaotao[(df_ctdaotao.hocky==1) | (df_ctdaotao.hocky==2)
                         |(df_ctdaotao.hocky==3)]
# nganh,hedt,khoahoc là những khóa sẽ kết với bảng dữ liệu chính
df_ctdaotao_selected=df_ctdaotao_selected.groupby(['nganh','khoadt','hedt'], as_index=False).aggregate({'sotc':'sum'})
df_ctdaotao_selected=df_ctdaotao_selected.rename(columns={"nganh": "chuyennganh", "khoadt": "khoahoc",
                                                         "hedt": "hedt_sv","sotc":"sotc_quydinh"})
df_ctdaotao_selected

In [ ]:
df_Final=pd.merge(df_Final,df_ctdaotao_selected,on=['chuyennganh','khoahoc','hedt_sv'],how='left')

In [ ]:
# TH ngành D480102 khóa 12 hệ CQUI không có ctdaotao trong hk1 và hk2
# Điền thiếu bằng số tín chỉ tối thiểu là 28 (=14+14)

sotc_toithieu=int(input())*2
sotc_toithieu

In [ ]:
df_Final['sotc_quydinh'] = df_Final['sotc_quydinh'].where(df_Final['sotc_quydinh'].notnull(),sotc_toithieu)
# df_Final.to_excel('df_Final3.xlsx')

# Điền thiếu các cột điểm

In [ ]:
# Tích hợp bảng dữ liệu để có điểm mà tính trung bình theo luật mới điền vào cho các môn thiếu.
df_dienthieu_diem
df_Final=pd.merge(df_Final,df_dienthieu_diem,on='masv',how='left')
# df_Final.to_excel('df_Final4.xlsx')

In [ ]:
print("Chon cach dien thieu diem: bam 1 (dtb) bam 2 (dien gia tri)")
Chon_diem=int(input())
if Chon_diem==2:
    print('dien gia tri diem')
    dien_diem=float(input())
print("Chon gia tri dien thieu cac thuoc tinh so con lai")
Chon_gt=float(input())

def dienthieu_monhoc(x):
    global Chon_diem
    global dien_diem
    global Chon_gt
       
    for i in range(Somon_canlay):
        mamh_='mamh_'+str(i+1)
        diemmon_='diemmon_'+str(i+1)
        sotcmon_='sotcmon_'+str(i+1)
        trangthaimon_='trangthaimon_'+str(i+1)
        sotc_ltmon_='sotc_ltmon_'+str(i+1)
        sotc_thmon_='sotc_thmon_'+str(i+1)
        hinhthucthmon_='hinhthucthmon_'+str(i+1)
        ngonngumon_='ngonngumon_'+str(i+1)
        khoaqlmon_='khoaqlmon_'+str(i+1)
        hedt_monhoc_='hedt_monhoc_'+str(i+1)
        hocky_mon_='hocky_mon_'+str(i+1)
        
        if x[diemmon_]=="":
            if Chon_diem==1:     
                if x['sotc_quydinh'] > x['Tong_sotc']:
                    x[diemmon_]=x['Diem*Tc']/x['sotc_quydinh']
                else:
                    x[diemmon_]=x['DTBnam']
            else:
                x[diemmon_]=dien_diem
        
        if x[sotcmon_]=="":
            x[sotcmon_]=Chon_gt
        if x[sotc_ltmon_]=="":
            x[sotc_ltmon_]=Chon_gt
        if x[sotc_thmon_]=="":
            x[sotc_thmon_]=Chon_gt
        
        if x[mamh_]=="":
            x[mamh_]='khongco'
        if x[trangthaimon_]=="":
            x[trangthaimon_]='khongco'
        if x[hinhthucthmon_]=="":
            x[hinhthucthmon_]='khongco'
        if x[ngonngumon_]=="":
            x[ngonngumon_]='khongco'
        if x[khoaqlmon_]=="":
            x[khoaqlmon_]='khongco'
        if x[hedt_monhoc_]=="":
            x[hedt_monhoc_]='khongco'
        if x[hocky_mon_]=="":
            x[hocky_mon_]='khongco'
        
    return x
        
        

In [ ]:
df_Final=df_Final.apply(dienthieu_monhoc,axis=1)


In [ ]:
# Xóa các thuộc tính không cần thiết
df_Final=df_Final.drop(columns='khoahoc')
df_Final=df_Final.drop(columns='sotc_quydinh')
df_Final=df_Final.drop(columns='Diem*Tc')
df_Final=df_Final.drop(columns='masv')
# Do trong quá trình tính toán Pandas thì các giá trị ko có là np.nan chứ không phải chuỗi rỗng
df_Final=df_Final.fillna(Chon_gt)

# (df_Final.isnull().sum()).to_excel('test.xlsx')


In [ ]:
chuoi=[]
chuoi.extend(['gioitinh','noisinh','cmnd_noicap'
              ,'lopsh','khoa','hedt_sv','chuyennganh','chuyennganh2','nganh_tt','dantoc','tongiao',
             'xuatthan','doituong','khuvuc'])
for i in range(Somon_canlay):
    mamh_='mamh_'+str(i+1)
    trangthaimon_='trangthaimon_'+str(i+1)
    hinhthucthmon_='hinhthucthmon_'+str(i+1)
    ngonngumon_='ngonngumon_'+str(i+1)
    khoaqlmon_='khoaqlmon_'+str(i+1)
    hedt_monhoc_='hedt_monhoc_'+str(i+1)
    hocky_mon_='hocky_mon_'+str(i+1)
    chuoi.extend([mamh_,trangthaimon_,
                        hinhthucthmon_,ngonngumon_,
                        khoaqlmon_,hedt_monhoc_,hocky_mon_])
def bodau_chuoi(x):
    for i in chuoi:
        x[i]=no_accent_vietnamese(x[i])
        
    return x
    
df_Final=df_Final.apply(bodau_chuoi,axis=1)

for i in chuoi:
    df_Final[i]=df_Final[i].str.lower().str.normalize('NFKC')

df_Final.columns= df_Final.columns.str.lower()    
df_Final

In [ ]:
df_Final['nhan_xuhuong'].value_counts()

In [ ]:
# df_Final.to_csv('df_nhanh13141516.csv')
df_Final.columns